# Extracción de reglas de asociación desde documentos utilizando algoritmo APRIORI

In [1]:
# Para importar el módulo con las funciones necesarias
import sys
sys.path.append(r"D:\Repos Github\AnaliticaTextual-Ejemplos") # la carpeta donde está el módulo

import utils # el módulo -> utils.py 

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules 
import pandas as pd # para creación y manejo de dataframes
import es_core_news_sm  # procesador de lenguaje con segmentación de frases, tokenización, lematización, etiquetas POS y dependencias sintácticas. Carga el modelo en español de Spacy
import regex # sirve para usar una librería de expresiones regulares más potente que la librería estándar 're' de Python.

In [7]:
# Esta función toma una lista de textos, extrae solamente los nombres (sustantivos y nombres propios) y luego hace un preprocesamiento clásico de NLP: stopwords → lemas → limpieza → compilación final.

def PreProcesarConNombres(textos):
    texto_limpio = []
    patron = r'(\w+)/(PROPN|NOUN)' #Busca tokens que terminen en /PROPN (nombre propio) o /NOUN (sustantivo).
    for texto in textos:  
        texto = utils.Lista_a_Oracion(ExtraerNombresLinea(patron,texto))
        texto = utils.EliminarStopwords(texto)    
        texto = utils.Lematizar(texto)     
        texto = utils.EliminaNumeroYPuntuacion(texto)      
        if len(texto)!=0:
          texto = regex.sub(' +', ' ', texto) # elimina espacios múltiples
          texto_limpio.append(texto)
    return(texto_limpio)

In [8]:
def ExtraerNombresLinea(patron,linea):
      texto      = linea.rstrip()
      etiquetado = utils.Etiquetar(texto)    
      ListaPalabras   = [w for (w,t) in regex.findall(patron,etiquetado)]
      return(ListaPalabras)

In [9]:
PATH = r"D:\Repos Github\AnaliticaTextual-Ejemplos\CORPUS\deportes"

In [11]:
nlp = es_core_news_sm.load()

In [16]:
# Crear corpus con textos de directorio apuntado por PATH
corpus,_ = utils.CrearCorpus(PATH) # Sólo tomo el corpus, no el corpus_id
textos = PreProcesarConNombres(corpus)
utils.CrearVSM(textos,"transacciones",False,True) #como modelo_idf = False ->Sólo TF, no TF-IDF, Modelo binario True -> matriz 0/1
#"Transacciones" -> nombre de la carpeta donde se guarda el modelo VSM
(doc_binarios,_,vocabulario) = utils.CargarModelo("transacciones")
# doc_binarios -> Matriz DTM, en este caso binaria
#_-> vacío, porque correspondería a los valores IDF (False)abs
#vocabulario-> vocabulario

In [17]:
# Para agilizar la extracción, elegir los 30 primeros términos por ahora
# De lo contrario:   LargoVoc = doc_binario.shape[1]
LargoVocab  = 30 

canasta = doc_binarios[:,0:LargoVocab] # tiene tamaño n_docs x 30

NombreColumnas = [ list(vocabulario)[x]  for x in range(0,LargoVocab)] # el orden del vocabulario es el mismo que el orden de la matriz doc_binarios

df = pd.DataFrame(canasta, columns = NombreColumnas) #Convertimos matriz numpy "canasta" en un DF de Pandas -> formato que espera mlxtend.apriori


itemsets = apriori(df, min_support = 0.005, use_colnames = True)
# min_support = 0.005 → solo te quedas con conjuntos de ítems que aparezcan en al menos el 0,5% de los documentos
# use_colnames = True → en vez de referirse a los ítems por número de columna, usa los nombres de las columnas (las palabras), lo que es mucho más interpretable.
# itemsets es un DataFrame que contiene, para cada conjunto frecuente:
    # la columna support
    # la columna itemsets (un frozenset de términos). conjunto (combinaciones) que no se puede modificar  

reglas = association_rules(itemsets, metric ="lift", min_threshold = 1) 
# generamos una regla de asociación a partir  de los conjuntos frecuente
# métrica lift. Nos quedamos con los que tienen un lift >=1 -> relación positiva entre antecedente y consecuente
# reglas tiene columnas como:
    # antecedents → conjunto de términos (lado izquierdo de la regla).
    # consequents → conjunto de términos (lado derecho).
    # support, confidence, lift, etc.
reglas_ordenadas = reglas.sort_values(['confidence', 'lift'], 
                                      ascending =[False, False])
# Ordenamos primero por confianza y, si empatan, por lift

print(reglas_ordenadas[['antecedents','consequents','confidence','lift']])


                antecedents              consequents  confidence       lift
8                 (joaquín)                (jornada)    1.000000  34.000000
9                 (jornada)                (joaquín)    1.000000  34.000000
10                (tarjeto)                   (tour)    1.000000  34.000000
11                   (tour)                (tarjeto)    1.000000  34.000000
14                  (texas)                 (bogeys)    1.000000  34.000000
15                 (bogeys)                  (texas)    1.000000  34.000000
18                 (unidos)                (niemann)    1.000000  34.000000
19                (niemann)                 (unidos)    1.000000  34.000000
22                (tarjeta)                    (pga)    1.000000  34.000000
23                    (pga)                (tarjeta)    1.000000  34.000000
32       (rondar, golfista)            (rendimiento)    1.000000  34.000000
33            (rendimiento)       (rondar, golfista)    1.000000  34.000000
36      (joa

C:\Users\srmjf\anaconda3\envs\atkinson\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [19]:
itemsets

,support,itemsets
0,0.058824,(golfista)
1,0.029412,(jornada)
2,0.029412,(tarjeto)
3,0.058824,(certamen)
4,0.029412,(texas)
5,0.058824,(estados)
6,0.029412,(unidos)
7,0.029412,(joaquín)
8,0.029412,(niemann)
9,0.029412,(corte)
